# Replica project

### The data

Stores the final deduplicated datasets and shows how the search can be done with pretrained embeddings

In [1]:
# loading the metadata
%load_ext autoreload
%autoreload 2

import pandas as pd

from torch import nn
import torchvision.models as models
from tqdm import tqdm
import torch
import pickle
from glob import glob
# from scipy import sparse
import sys
import os.path
from sklearn import preprocessing
from os import path


sys.path.insert(0, "../model/")
from utils import *
from model_replica import ReplicaNet


path = '/home/guhennec/scratch/2021_Cini/TopologicalAnalysis_Cini/data/'
data_dir = '/scratch/students/schaerf/'
replica_dir = '/mnt/project_replica/datasets/cini/'
wga_dir = '/mnt/project_replica/datasets/'


path = '../data/'
data_dir = '../data/'
replica_dir = '../data/'
wga_dir = '../data/'




In [2]:
metadata = pd.read_csv(path + 'Cini_20210811.csv', sep=';')
metadata['path'] = replica_dir + metadata['Drawer'].astype(str) + '/' + metadata['Drawer'].astype(str) + '_' + metadata['ImageNumber'].astype(str) + '.jpg'
metadata.head(5)

C:\Users\ludov\AppData\Local\Temp/ipykernel_13936/30539250.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(path + 'Cini_20210811.csv', sep=';')


,Drawer,ImageNumber,Author,Description,AuthorOriginal,AuthorULANLabel,AuthorULAN,AuthorURL,AuthorComplemented,AuthorComplement,...,AuthorBirthLong,AuthorBirthLat,AuthorDeathLong,AuthorDeathLat,AuthorBirthCity,AuthorDeathCity,uid,CardboardURL,ImageURL,path
0,1A,1,ALLEGRINI Francesco,tca con penna acquarellata 148 x 203 mm,"ALLEGRINI, Francesco","Allegrini, Francesco",ulan:500115272,http://vocab.getty.edu/page/ulan/500115272,0,No complement,...,NaN,NaN,NaN,NaN,NaN,NaN,253993c139284a45be233a13121ddeeb,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_1....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,../data/1A/1A_1.jpg
1,1A,2,BAROCCI Federico,Studio di figura penna acquarellata (carboncin...,"BAROCCI, Federico","Barocci, Federico",ulan:500115210,http://vocab.getty.edu/page/ulan/500115210,0,No complement,...,12.633333,43.716667,12.633333,43.716667,Urbino,Urbino,1323356994c24635a11fdcd9d5f9284a,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_2....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,../data/1A/1A_2.jpg
2,1A,3,BASSANO Leandro,Scene penna su carta blu 171 x 219 mm,"BASSANO, Leandro","Bassano, Leandro",ulan:500015945,http://vocab.getty.edu/page/ulan/500015945,0,No complement,...,11.733333,45.766667,12.326667,45.438611,Bassano del Grappa,Venice,a4268385f6384e61a3dd092bc6b8c083,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_3....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,../data/1A/1A_3.jpg
3,1A,4,CAMPIGLI Massimo,Ragazza su una scala gessetto su carta bruna g...,"CAMPIGLI, Massimo","Campigli, Massimo",ulan:500029770,http://vocab.getty.edu/page/ulan/500029770,0,No complement,...,13.400000,52.516667,6.639811,43.269316,Berlin,Var,550f368cdb4442aab4d5e2ada702d6ad,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_4....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,../data/1A/1A_4.jpg
4,1A,5,CARRACCI A attr,Madonna e Santi penna 291 x 205 mm,"CARRACCI, A. attr.",NaN,NaN,NaN,1,attr,...,NaN,NaN,NaN,NaN,NaN,NaN,6047de5547b643cb87491be925748bee,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_5....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,../data/1A/1A_5.jpg


In [3]:
metadata.columns

Index(['Drawer', 'ImageNumber', 'Author', 'Description', 'AuthorOriginal',
       'AuthorULANLabel', 'AuthorULAN', 'AuthorURL', 'AuthorComplemented',
       'AuthorComplement', 'AuthorNeighbour', 'AuthorNeighbourhood',
       'SimpleCollection', 'CiniCollection', 'CiniNumber', 'CiniTime', 'City',
       'Country', 'FondoStamp', 'Institution', 'Reference', 'AuthorGender',
       'AuthorNationality', 'BeginDate', 'EndDate', 'BiographyLabel',
       'AuthorBirth', 'AuthorDeath', 'AuthorBirthLong', 'AuthorBirthLat',
       'AuthorDeathLong', 'AuthorDeathLat', 'AuthorBirthCity',
       'AuthorDeathCity', 'uid', 'CardboardURL', 'ImageURL', 'path'],
      dtype='object')

## Full data

In [4]:
#metadata_2 = pd.read_excel(wga_dir + 'catalog.xls') #no uid
#print(metadata_2)

#metadata_2 = pd.read_csv(wga_dir + 'index.csv',).drop('Unnamed: 0', axis=1)
#metadata_2.columns = ['Author', 'AuthorBirth', 'Description', 'path', 'BeginDate', 'date_num', 'height', 'width', 'is_color', 'uid']

with open(data_dir + 'image_data_2022_04_28.pkl', 'rb') as infile:
    metadata_2 = pickle.load(infile)


metadata_2.head()


,uid,added,source,path,o_uid,o_title,o_date_begin,o_date_end,o_attribution,o_metadata
0,9515070c-b367-4ee5-b7e0-4eb8cacbcae7,2017-07-11 16:42:41.307381+00:00,cini,cini/2A/2A_1.jpg,c37fd54e-be01-495c-b5ac-0ef08a58fb9f,Veduta dell'Etna e di Taormina.\nGuazzo: 36 x ...,1771.0,1794.0,Fondazione Giorgio Cini,"{'City': 'CAEN', 'Author': 'CHATELET Claude Lo..."
1,a91b058d-a3f4-4e14-a67a-8e710754be5d,2017-07-11 16:42:41.445597+00:00,cini,cini/2A/2A_2.jpg,2b05a698-3e3b-4be6-8aaa-c8d20b3217b5,Veduta del teatro di Taormina.\nGuazzo: 350 x ...,1771.0,1794.0,Fondazione Giorgio Cini,"{'City': 'CAEN', 'Author': 'CHATELET Claude Lo..."
2,1160b942-4c34-4b9c-88a8-2c1d255c5961,2017-07-11 16:42:41.543159+00:00,cini,cini/2A/2A_3.jpg,bc615e17-c55e-48f0-bc3e-e01b325b8b7e,Nettuno e Anfitride.\nmm. 75 x 90,1528.0,1563.0,Fondazione Giorgio Cini,"{'City': 'C H A M B E RY', 'Author': 'SCHIAVON..."
3,a003eb3f-2667-4763-b9a1-1c2daae8b5a1,2017-07-11 16:42:41.631198+00:00,cini,cini/2A/2A_4.jpg,7c1484cf-7069-4cdd-bd3d-2a00935d51a8,Veduta di Nepi.\nMatita: 435 x 385\nT.to.,1815.0,1871.0,Fondazione Giorgio Cini,"{'City': 'C H A RTR ES', 'Author': 'BERTIN Edo..."
4,9903179a-30dc-4a7b-8b39-7456d480e2e2,2017-07-11 16:42:41.742387+00:00,cini,cini/2A/2A_5.jpg,d90c949f-9db1-415b-b5b8-1cb5c0f0f46d,Madonna con Bambino in gloria d'Angeli,1574.0,1629.0,Fondazione Giorgio Cini,"{'City': 'CH A N TIL L Y', 'Author': 'ALIENSE ..."


In [5]:
metadata_2.reset_index()['o_metadata'][0]

{'City': 'CAEN',
 'Author': 'CHATELET Claude Louis',
 'AuthorId': '<a href="http://vocab.getty.edu/page/ulan/500013578">ulan:500013578</a>',
 'CiniTime': '1961',
 'AuthorRaw': 'CHATELET, Claude Louis',
 'DateRange': '1771-1794',
 'Reference': "M. L'Italia vista dai pittori francesi del XVIII e XIX secolo, (1961), Cat. n. 63",
 'CiniNumber': 'N 2104',
 'Description': "Veduta dell'Etna e di Taormina.\nGuazzo: 36 x 51.\n1780.",
 'Institution': 'MUSEE DES BEAUX ARTS'}

In [6]:
metadata_2 = metadata_2[metadata_2['source'] == 'wga']   
metadata_2.reset_index()['o_metadata'][0]

{'URL': 'http://www.wga.hu/html/a/aachen/allegory.html',
 'DATE': 1598,
 'FORM': 'painting',
 'TYPE': 'mythological',
 'TITLE': 'Allegory',
 'AUTHOR': 'AACHEN, Hans von',
 'SCHOOL': 'German',
 'LOCATION': 'Alte Pinakothek, Munich',
 'TIMELINE': '1601-1650',
 'BORN-DIED': '(b. 1552, Köln, d. 1615, Praha)',
 'TECHNIQUE': 'Oil on copper, 56 x 47 cm'}

In [7]:
metadata_2['AuthorOriginal'] = metadata_2['o_metadata'].apply(lambda x: x['AUTHOR'])
metadata_2['AuthorBirth'] = metadata_2['o_metadata'].apply(lambda x: x['BORN-DIED'])
metadata_2['Description'] = metadata_2['o_metadata'].apply(lambda x: x['TITLE'])
metadata_2['BeginDate'] = metadata_2['o_date_begin']
metadata_2['ImageURL'] = metadata_2['o_metadata'].apply(lambda x: x['URL'])
metadata_2['Type'] = metadata_2['o_metadata'].apply(lambda x: x['TYPE'])


metadata_2['uid'] = metadata_2['uid'].astype(str).apply(lambda x: x.replace('-', ''))
metadata_2['path'] = wga_dir + metadata_2['path']

metadata_2 = metadata_2[['AuthorOriginal', 'AuthorBirth', 'Description', 'path', 'BeginDate', 'ImageURL', 'Type', 'uid',]]
metadata_2.head()


,AuthorOriginal,AuthorBirth,Description,path,BeginDate,ImageURL,Type,uid
10789,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Allegory,../data/WGA/images/a/aachen/allegory.jpg,1598.0,http://www.wga.hu/html/a/aachen/allegory.html,mythological,2355b564f41e41a09bf6ae2a503e152a
10790,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Bacchus, Ceres and Cupid",../data/WGA/images/a/aachen/bacchus.jpg,1601.0,http://www.wga.hu/html/a/aachen/bacchus.html,mythological,d57ed156ca104cbcb52d0ab4f76c84a1
10791,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Joking Couple,../data/WGA/images/a/aachen/j_couple.jpg,1601.0,http://www.wga.hu/html/a/aachen/j_couple.html,genre,d51bd5c5a7b1440587b437d428e270a6
10792,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Portrait of Emperor Rudolf II,../data/WGA/images/a/aachen/rudolf2.jpg,1590.0,http://www.wga.hu/html/a/aachen/rudolf2.html,portrait,5d7c0617643d4f7eb58cf38df03b8eb2
10793,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Self-Portrait with a Glass of Wine,../data/WGA/images/a/aachen/selfport.jpg,1591.0,http://www.wga.hu/html/a/aachen/selfport.html,portrait,b4de719d8ca84709a1fffafe7bb89db0


In [8]:
metadata_2['Type'].value_counts()

religious       16935
portrait         5619
landscape        4335
mythological     4033
genre            2860
other            2349
still-life       1402
study            1283
historical       1161
interior          649
Name: Type, dtype: int64

In [9]:
le = preprocessing.LabelEncoder()

metadata_2['TypeN'] = le.fit_transform(metadata_2['Type'])
metadata_2.head()

,AuthorOriginal,AuthorBirth,Description,path,BeginDate,ImageURL,Type,uid,TypeN
10789,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Allegory,../data/WGA/images/a/aachen/allegory.jpg,1598.0,http://www.wga.hu/html/a/aachen/allegory.html,mythological,2355b564f41e41a09bf6ae2a503e152a,4
10790,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Bacchus, Ceres and Cupid",../data/WGA/images/a/aachen/bacchus.jpg,1601.0,http://www.wga.hu/html/a/aachen/bacchus.html,mythological,d57ed156ca104cbcb52d0ab4f76c84a1,4
10791,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Joking Couple,../data/WGA/images/a/aachen/j_couple.jpg,1601.0,http://www.wga.hu/html/a/aachen/j_couple.html,genre,d51bd5c5a7b1440587b437d428e270a6,0
10792,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Portrait of Emperor Rudolf II,../data/WGA/images/a/aachen/rudolf2.jpg,1590.0,http://www.wga.hu/html/a/aachen/rudolf2.html,portrait,5d7c0617643d4f7eb58cf38df03b8eb2,6
10793,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Self-Portrait with a Glass of Wine,../data/WGA/images/a/aachen/selfport.jpg,1591.0,http://www.wga.hu/html/a/aachen/selfport.html,portrait,b4de719d8ca84709a1fffafe7bb89db0,6


In [10]:
metadata_2 = metadata_2[metadata_2['path'].notnull()]

In [11]:
# metadata_2.to_csv(data_dir + 'train_type.csv')

In [12]:
metadata_2[metadata_2['uid'] == 'fd08dd6652684417a7809bf727111d6f']

,AuthorOriginal,AuthorBirth,Description,path,BeginDate,ImageURL,Type,uid,TypeN
30114,LEONARDO da Vinci,"(b. 1452, Vinci, d. 1519, Cloux, near Amboise)",Virgin of the Rocks,../data/WGA/images/l/leonardo/02/3virg_l.jpg,1495.0,http://www.wga.hu/html/l/leonardo/02/3virg_l.html,religious,fd08dd6652684417a7809bf727111d6f,7


In [13]:
#metadata_2[metadata_2['AuthorBirth'] == '(b. 1794, St. Petersburg, d. 1855, St. Petersburg)']

In [14]:
metadata_tot = pd.concat([metadata[['AuthorOriginal', 'AuthorBirth', 'Description', 'path', 'BeginDate', 'uid', 'ImageURL', ]], 
                          metadata_2[['AuthorOriginal', 'AuthorBirth', 'Description', 'path', 'BeginDate', 'uid', 'ImageURL', 'Type',]]], axis=0)
print(metadata_tot.shape)
metadata_tot.head()

(370628, 8)


,AuthorOriginal,AuthorBirth,Description,path,BeginDate,uid,ImageURL,Type
0,"ALLEGRINI, Francesco",1610.0,tca con penna acquarellata 148 x 203 mm,../data/1A/1A_1.jpg,1628.0,253993c139284a45be233a13121ddeeb,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,NaN
1,"BAROCCI, Federico",1528.0,Studio di figura penna acquarellata (carboncin...,../data/1A/1A_2.jpg,1546.0,1323356994c24635a11fdcd9d5f9284a,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,NaN
2,"BASSANO, Leandro",1557.0,Scene penna su carta blu 171 x 219 mm,../data/1A/1A_3.jpg,1575.0,a4268385f6384e61a3dd092bc6b8c083,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,NaN
3,"CAMPIGLI, Massimo",1895.0,Ragazza su una scala gessetto su carta bruna g...,../data/1A/1A_4.jpg,1913.0,550f368cdb4442aab4d5e2ada702d6ad,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,NaN
4,"CARRACCI, A. attr.",NaN,Madonna e Santi penna 291 x 205 mm,../data/1A/1A_5.jpg,NaN,6047de5547b643cb87491be925748bee,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,NaN


In [15]:
metadata_tot['uid'].nunique(), metadata_tot.shape

(370106, (370628, 8))

In [16]:
# dropping duplicates

metadata_tot = metadata_tot.groupby('uid').first().reset_index()

In [17]:
with open(path + 'save_link_data_2018_08_02.pkl', 'rb') as f:
    morpho_graph_complete = pickle.load(f)

# contains uid	img1	img2	type	annotated
morpho_graph_complete.head()

,uid,img1,img2,type,annotated
0,0f84770be893475581bebd8c0ce95704,4b6e5a18012e4974a43f6f652c9cad78,e3329a91a266412fb1a0975d70894178,POSITIVE,2017-07-19 15:02:50.997512+00:00
1,5ade49fc2dc649b9947496a41e2decdb,384b9f017d5f43ba9dcfd92f2af830c4,e3329a91a266412fb1a0975d70894178,POSITIVE,2017-07-19 15:02:51.238615+00:00
2,bf4d99eaf7da4bd486f9007c90c177a5,94fe74bcebf9442fb5cd9a0327590a84,726f47fb5bd94c5db8069334cc58020e,POSITIVE,2017-07-20 12:18:23.705228+00:00
3,b29ec3c83c5641f9b6afadb2409b6746,922fe92d61e744b1a9462c848a916bcf,726f47fb5bd94c5db8069334cc58020e,POSITIVE,2017-07-20 12:18:24.021703+00:00
4,ab27b55997b94bef9cca122829e9f64d,94fe74bcebf9442fb5cd9a0327590a84,922fe92d61e744b1a9462c848a916bcf,POSITIVE,2017-07-20 12:18:23.873097+00:00


In [18]:
with open(data_dir + 'morphograph_update.pkl', 'rb') as f:
    morpho_graph_update = pickle.load(f)

# contains uid	img1	img2	type	annotated
morpho_graph_update.head()

,uid,img1,img2,type,annotated
0,af99c7f1401e4288bacca135ad916c2c,af99c7f1401e42889d36614b50f48fcd,d1c5eedffa3c42eabacca135ad916c2c,POSITIVE,08-04-2022_16:50:31
0,e348cd2149fd4fb18b79415e1dea8630,e348cd2149fd4fb1be2fee761d616c42,8286397109c440f48b79415e1dea8630,POSITIVE,08-04-2022_18:30:49
0,d50f7a340d7148ec8a3aaba293ba7ed9,d50f7a340d7148ec807ce12bc35da523,faa0c51a85ff406c8a3aaba293ba7ed9,POSITIVE,08-04-2022_18:31:07
0,df27d9eff7a74e34bf04bf87c65d128d,df27d9eff7a74e3499359c4b00883d5e,1db9c995dd594309bf04bf87c65d128d,POSITIVE,08-04-2022_18:53:58
0,bb7283d826784f74a52095f2f8496379,bb7283d826784f74b29fe4a855c9b16c,e186ba530c7149c7a52095f2f8496379,POSITIVE,08-04-2022_18:54:32


In [20]:
morpho_graph_clusters = pd.read_csv(data_dir + 'morphograph_clusters.csv')
morpho_graph_clusters['uid'] = morpho_graph_clusters['uid_connection']
morpho_graph_clusters['annotated'] = morpho_graph_clusters['date']
morpho_graph_clusters = morpho_graph_clusters.drop(columns=['uid_connection', 'date','cluster_file', 'cluster'])
morpho_graph_clusters.head()

,img1,img2,type,uid,annotated
0,810270aed30c4843a5c637c8f74ce640,2557e969c9d644dba3dd15f4508c3e84,POSITIVE,810270aed30c4843a3dd15f4508c3e84,06-05-2022_19:24:28
1,2557e969c9d644dba3dd15f4508c3e84,810270aed30c4843a5c637c8f74ce640,POSITIVE,2557e969c9d644dba5c637c8f74ce640,06-05-2022_19:24:28
2,788e9f6a64f4432ba09da66bbb60bf93,d6e13eefc497491ca8d76cb92f0ad745,POSITIVE,788e9f6a64f4432ba8d76cb92f0ad745,06-05-2022_19:29:13
3,d6e13eefc497491ca8d76cb92f0ad745,788e9f6a64f4432ba09da66bbb60bf93,POSITIVE,d6e13eefc497491ca09da66bbb60bf93,06-05-2022_19:29:13
4,14a738ba9b91454a997738105ccf5c9b,05f5c4e0325f40019eeb0bb542c1f692,POSITIVE,14a738ba9b91454a9eeb0bb542c1f692,06-05-2022_19:29:43


In [21]:
morpho_graph_complete = pd.concat([morpho_graph_complete, morpho_graph_update, morpho_graph_clusters], axis=0)
morpho_graph_complete.shape

(120922, 5)

In [22]:
morpho_graph_complete[morpho_graph_complete['img1'].str.startswith('fd08dd6652684417a7809bf727111d6f')] # virgin of the rocks

,uid,img1,img2,type,annotated
1550,7c638674ef92402b9334047c3c55ab75,fd08dd6652684417a7809bf727111d6f,7723291f006c47439fbe9f44053f79de,POSITIVE,2016-05-31 09:50:44+00:00
1551,352a26b9de3142c89f8e3a1f8f1fa743,fd08dd6652684417a7809bf727111d6f,56cde7613a5044a9bf57af9d80691c5b,POSITIVE,2016-05-31 09:50:44+00:00
1553,d47ac9d7b80b43c6a9d7ba45e8cb6db1,fd08dd6652684417a7809bf727111d6f,8bd5300d3a69411b9c799b439d7ce8fa,POSITIVE,2016-09-13 14:22:55+00:00
1568,b2003df6d9074acdb041457f311cbd0b,fd08dd6652684417a7809bf727111d6f,a3f1b0cbcbc54682a8f393d5cc48c85f,POSITIVE,2016-05-31 09:50:44+00:00
1576,097ef90463ae4530a734a8b3f4d28c0f,fd08dd6652684417a7809bf727111d6f,8070848b55034bb5a010c420540b93ab,POSITIVE,2016-05-31 09:50:44+00:00
2721,905213dd0c7a485abd73308db2dae838,fd08dd6652684417a7809bf727111d6f,0e5f69c372a742778f0e6fb84df884be,POSITIVE,2016-09-13 14:22:55+00:00
2724,7dafd1f5537d4a329bda39e0736b70ed,fd08dd6652684417a7809bf727111d6f,380a3dd4f16447ce8d7dc422bba2f191,POSITIVE,2016-09-13 14:22:55+00:00
7805,3746c2e181244adeac8f53cc7c28a807,fd08dd6652684417a7809bf727111d6f,c0d9011fda40474485acb7a6a46bf070,DUPLICATE,2018-05-05 17:48:45.407263+00:00
8049,44d1891e335343a6b8f6ac5b4abfcc69,fd08dd6652684417a7809bf727111d6f,ff57569da2134eec861c70f4ac081db9,POSITIVE,2018-05-08 08:52:19.793174+00:00


In [23]:
morpho_graph_complete['type'].value_counts() # does positive mean similar?

DUPLICATE        114283
POSITIVE           6466
NEGATIVE             86
NON-DUPLICATE        60
PROPOSAL             27
Name: type, dtype: int64

In [24]:
morpho_graph_complete['uid'].nunique(), morpho_graph_complete.shape

(120916, (120922, 5))

In [25]:
positives = get_train_test_split(metadata_tot, morpho_graph_complete)
positives.set.value_counts()

(8867, 14)


train    5418
val      2021
test     1428
Name: set, dtype: int64

In [26]:
positives.columns

Index(['index', 'uid', 'AuthorOriginal', 'AuthorBirth', 'Description', 'path',
       'BeginDate', 'ImageURL', 'Type', 'uid_connection', 'img1', 'img2',
       'type', 'annotated', 'cluster', 'set'],
      dtype='object')

In [27]:
positives = positives.groupby('uid_connection').first().reset_index()
print(positives.shape)

(5133, 16)


In [28]:
positive = positives.groupby('uid').last().reset_index()

In [29]:
positive.to_csv(data_dir + 'morpho_dataset.csv')

In [29]:
# positives[positives['Description'].fillna('').str.contains("L' Adultera.")]

In [30]:
# positives[positives['Author'].fillna('').str.contains('SPAGNA Giovanni')]

## Getting a subset of the data

In order to select a subset of the data, we would like to vary spatially and temporally, for it to be suitable to the data in the morphograph and to be comprehensive of what we want to study.


In [31]:
metadata_tot['BeginDate'].fillna(0).astype(str).str.strip('c. ').str[:-2].str.strip('-')

0         1529
1         1600
2         1501
3         1444
4         1623
          ... 
370101       0
370102       0
370103    1659
370104    1762
370105    1323
Name: BeginDate, Length: 370106, dtype: object

In [32]:
#subset_max = metadata[metadata['BeginDate'] > 1550][metadata['BeginDate'] < 1650][metadata['AuthorDeathCity'].isin(['Parma', 'Bologna', 'Ravenna', 'Verona', 'Ferrara'])].reset_index()

metadata_tot['BeginDate'] = [int(met) if (len(met) == 4) and ('/' not in met) else 0 for met in metadata_tot['BeginDate'].fillna(0).astype(str).str.strip('c. ').str[:-2].str.strip('-, ')]
metadata_tot.shape

(370106, 8)

In [33]:
subset_max = metadata_tot[metadata_tot['BeginDate'] > 1400][metadata_tot['BeginDate'] < 1700]#[metadata['AuthorDeathCity'].isin(['Parma', 'Bologna', 'Ravenna', 'Verona', 'Ferrara'])].reset_index()
subset_max.shape

/tmp/ipykernel_570055/830820037.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subset_max = metadata_tot[metadata_tot['BeginDate'] > 1400][metadata_tot['BeginDate'] < 1700]#[metadata['AuthorDeathCity'].isin(['Parma', 'Bologna', 'Ravenna', 'Verona', 'Ferrara'])].reset_index()


(155004, 8)

In [34]:
subset_max = remove_duplicates(subset_max, morpho_graph_complete)
subset_max.shape

(122969, 15)

In [35]:
subset_max = subset_max.sort_values('uid_connection').groupby(['Author', 'Description']).first().reset_index()

In [36]:
subset_max[subset_max['Author'] == 'BASSANO (bottega di)'][subset_max['Description'].str.contains('Presepio')]

/tmp/ipykernel_570055/686436682.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subset_max[subset_max['Author'] == 'BASSANO (bottega di)'][subset_max['Description'].str.contains('Presepio')]


,Author,Description,cluster,index,uid,AuthorBirth,path,BeginDate,ImageURL,Type,uid_connection,img1,img2,type,annotated
7616,BASSANO (bottega di),Presepio.,cb61e9056bbf45c8b3d95dedb8ceafca,143417,cb61e9056bbf45c8b3d95dedb8ceafca,1505.0,/mnt/project_replica/datasets/cini/100A/100A_8...,1523.0,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,None,None,None,None,None,NaT


In [37]:
#subset_max['Description'].value_counts().head(30)

In [38]:
# list_downloaded = ['/'.join(file.split('/')[-2:]) for file in glob('/mnt/project_replica/datasets/cini/*/*')]
# list_downloaded[0]

In [39]:
# len(list_downloaded)

In [40]:
# list_downloaded_2 = ['/'.join(file.split('/')[-2:]) for file in glob(wga_dir + 'WGA/images/*/*/*/*')]
# list_downloaded_2[0]

In [41]:
# len(list_downloaded_2)

In [42]:
#uniques = subset_max[subset_max['uid'].isin(list_downloaded)].groupby('uid').first().reset_index()
#subset_max['uid2'] = subset_max['Drawer'].astype(str) + '/' + subset_max['Drawer'].astype(str) + '_' + subset_max['ImageNumber'].astype(str) + '.jpg'

#uniques = subset_max[subset_max['uid2'].isin(list_downloaded)].groupby('uid').first().reset_index()
uniques = subset_max[subset_max['path'].notnull()].groupby('uid').first().reset_index()

print(uniques.shape)

(108706, 15)


In [43]:
subset_max['path'].nunique()

108706

In [44]:
#uniques['Desc'] = uniques['Description'].str.lower().str.split()
#words = uniques.explode('Desc').reset_index()['Desc'].value_counts()
#print(words[words.index.str.len() > 4][words > 100].to_string())

In [45]:
uniques = uniques[uniques['Description'].notnull()]

In [46]:
to_remove = ['tavolo', 'facciata', 'paesaggio', 'esterno', 'pag.', 'soffitto',
             'sedia', 'veduta', 'scorcio', 'capitell', 'architettonic',
             'fontana', 'edifici', 'libr', 'architettonic']
uniques = uniques[~uniques['Description'].str.lower().str.contains(r'|'.join(to_remove))]
uniques.shape

(99464, 15)

In [47]:
uniques['uid_connection'] = uniques['uid_connection'].fillna('Not uid connection') 

In [48]:
uniques[uniques['path'].str.contains('WGA')].shape

(17754, 15)

In [49]:
uniques[uniques['ImageURL'].str.contains('cini')].shape

(81724, 15)

In [50]:
data = pd.concat([uniques[uniques['ImageURL'].str.contains('cini')], positives], axis=0).reset_index().groupby(['uid', 'uid_connection']).last().reset_index()
# data_sample = pd.concat([uniques.sample(10000), positives], axis=0).reset_index()#.groupby('uid').first().reset_index()
print(data.shape)

(86820, 17)


In [90]:
# data_type = pd.concat([uniques[uniques['Type'].isin(['religious', 'portrait', 'mythological', 'genre'])], positives], axis=0).reset_index()
# print(data_type.shape)
# data_type.to_csv(data_dir + 'wga_data_type.csv')


In [91]:
# data_sample = pd.concat([uniques[uniques['Type'].isna()].sample(20000), data_type], axis=0).drop(columns='level_0').reset_index()#.groupby('uid').first().reset_index()
# data_sample.shape
# data_type.to_csv(data_dir + 'data_wga_cini_45000.csv')


In [51]:
uid2path = {}

for uid, path in tqdm(zip(data['uid'].unique(), data['path'].unique())):
    uid2path[uid] = path

with open(data_dir + 'uid2path.pkl', 'wb') as outfile:
    pickle.dump(uid2path, outfile)

83190it [00:00, 1072616.06it/s]


In [52]:
data['uid'].nunique(), data['path'].nunique()

(83190, 83190)

In [56]:
data[data['type'].notnull()].sample(5)

,uid,uid_connection,level_0,Author,Description,cluster,index,AuthorBirth,path,BeginDate,ImageURL,Type,img1,img2,type,annotated,set
76076,d93fc67e945144e29d5150b148e16193,024bfea419b64c00b73034e69ddbb07a,42,"OGGIONO, Marco d'",Virgin and Child Enthroned with Saints,173,3799,"(b. ca. 1467, Milano, d. 1524, Milano)",/mnt/project_replica/datasets/WGA/images/o/ogg...,1519.0,http://www.wga.hu/html/o/oggiono/virgin_c.html,religious,d93fc67e945144e29d5150b148e16193,e832838713ce4549a45936560edb3187,POSITIVE,2016-06-16 14:37:21+00:00,train
408,00daa61de7c94200a7c2a3ba4a331caf,f4a352043fa940a09a69e5a59a9982be,524,SEBASTIAN DEL PIOMBO (attr),"La Sacra Famiglia, donatore e i Ss. Caterina e...",16181,650,1480.0,/mnt/project_replica/datasets/cini/157A/157A_2...,1498.0,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,None,00daa61de7c94200a7c2a3ba4a331caf,2c09b8b59e504081b730a795463c7a00,DUPLICATE,2018-07-19 17:22:58.620949+00:00,None
2987,06963fbb80914abe8187bae58fe64807,a2de135a5b7f43c58483de07365c07b7,3839,GHIRLANDAIO Domenico,Natività,19328,4725,1449.0,/mnt/project_replica/datasets/cini/50C/50C_835...,1467.0,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,None,06963fbb80914abe8187bae58fe64807,b1409de4c370480fb8143737fd304c2a,DUPLICATE,2018-07-19 17:27:05.770597+00:00,None
67182,bab253b84134466692fa5325627d62e1,30cdf8d3adb743c3b89ac30250c8e511,84137,RAFFAELLO Sanzio,Il Parnaso. Part.,9233,131866,1483.0,/mnt/project_replica/datasets/cini/120C/120C_3...,1501.0,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,None,bab253b84134466692fa5325627d62e1,debd5cacdb494fa3b9c35c8475400b98,DUPLICATE,2018-07-19 17:15:38.504996+00:00,None
30358,4aad5fa8c61a41febbb8e9380f85091d,770ae869723c4f39aa6be7e5e53f3077,38251,RAFFAELLO Sanzio,Ritratto di Angelo Doni.,2055,53167,1483.0,/mnt/project_replica/datasets/cini/137A/137A_8...,1501.0,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,portrait,4aad5fa8c61a41febbb8e9380f85091d,8ea0cb34bda04454916b5a162c48b17c,DUPLICATE,2018-07-19 17:10:17.773477+00:00,None


In [54]:
data[data['set'].isin(['train', 'test', 'val'])].shape, #data_sample[data_sample['set'].isin(['train', 'test', 'val'])].shape


((5096, 17),)

In [55]:
# data_sample.to_csv(data_dir + 'dedup_data_sample_wga.csv')
data.to_csv(data_dir + 'data.csv')


In [25]:
#data.to_csv(data_dir + 'full_data.csv')
# data_sample.to_csv(data_dir + 'dedup_data_sample.csv')
# data.to_csv(data_dir + 'dedup_data.csv')


### Creating embeddings of all the images

In [6]:
model = models.resnet50(pretrained=True)
#print(model)

model = ReplicaNet('resnext-101', 'cuda')

In [29]:
# newmodel = torch.nn.Sequential(
#     *(list(model.children())[:-2]), nn.AdaptiveMaxPool2d((1,1))
# )
#print(newmodel)


In [16]:
it = 10462
print(data['path'][it])
img = preprocess_image_test(data['path'][it])

img.shape

/mnt/project_replica/datasets/WGA/images/t/tiziano/03_1530s/1madonn.jpg


torch.Size([1, 3, 480, 480])

In [19]:
embedding = get_embedding(img, model, device='cuda')
print(embedding)

[0.02288655 0.01400073 0.0181444  ... 0.01918198 0.02341286 0.01433349]


In [35]:
def catch(path, model, device, resolution):
    try:
        return get_embedding(preprocess_image_test(path, resolution=resolution), model, device=device)
    except Exception as e:
        print(e)
        return 0

In [39]:
catch(data['path'][460], model, 'cuda', 320)

output with shape [1, 320, 320] doesn't match the broadcast shape [3, 320, 320]


0

In [ ]:
embeddings = [[uid, catch(path, model, 'cuda', 320)] for uid,path in tqdm(zip(data['uid'].unique(), data['path'].unique()))]

In [32]:
embeddings_new = get_lower_dimension(embeddings)

In [33]:
embeddings[0].shape, embeddings_new[0].shape 

((2048,), (100,))

### Finding most similar embeddings

In [34]:
tree = make_tree_list(embeddings_new)

In [60]:
train_test = data[data['set'].notnull()].drop(columns=['level_0']).reset_index()

Cs = []
for i in tqdm(range(train_test.shape[0])):
    list_theo = list(train_test[train_test['img1'] == train_test['uid'][i]]['img2']) + list(train_test[train_test['img2'] == train_test['uid'][i]]['img1']) + [train_test['uid'][i]]
    list_ = find_most_similar_list(train_test['uid'][i], tree, embeddings_new, list(data['uid'].unique()), list_theo)
    Cs.append(list_)

len(Cs)


100%|██████████| 2862/2862 [00:10<00:00, 286.12it/s]


2862

In [58]:
print(train_test[train_test['img1'].isin(list_downloaded)].shape)
print(train_test[train_test['img2'].isin(list_downloaded)].shape)

(2299, 47)
(2256, 47)


In [61]:
train_test['C'] = Cs

train_test = train_test[train_test['img1'].isin(list_downloaded)]
train_test = train_test[train_test['img2'].isin(list_downloaded)]

final = train_test[['img1', 'img2', 'C', 'set']].explode('C')
final.columns = ['A', 'B', 'C', 'set']
print(final.shape)
final.sample(5)

(31709, 4)


,A,B,C,set
1771,6fcf1af179fa47488d40564e80856a1e,e100fa4b1d1e4abab65256ac310ab543,6bf773f2e04f4c919ccd6f0b76df2b15,train
1539,ef25d7d6549547b59945a61ae6849d3d,272cd39493464a878264554be785e9db,92fc36e4d04743ec97e441342acd29f1,test
63,59fb95590dda4ff58e63dc0df43e70c3,a043a8ccb7994ef68179ad66929143af,756a2003514740448ffb37eec9351177,train
289,383cd750b1cc425089ff3cf3d2c81df1,8b47beb1f1a6438993f3988e880cfa9c,45d519d9ba884c948b2a0abf1a692599,test
581,3d7b289477ca4765a6932bd85cf51fec,246ca4d9a3324ad2aad8ec6a4e1c6769,4ced1a6287c1417db8b41e79e1a04c79,test


In [62]:
final[final['C'].isin(list_downloaded)].shape

(31709, 4)

In [63]:
final[final['set'] == 'train'].reset_index().to_csv(data_dir + 'abc_train.csv')
final[final['set'] == 'test'].reset_index().to_csv(data_dir + 'abc_test.csv')
